In [105]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib import request
from urllib import error
from tqdm._tqdm_notebook import tqdm_notebook
from http.client import RemoteDisconnected

In [29]:
df1 = pd.read_csv('/home/pacbio/Heat Map/argonaute_final.tsv',header=0,sep='\t')

Uniprotでタンパク質IDを検索してドメインを取得する

In [30]:
def DomainName(j):
    url = "https://www.uniprot.org/uniprot/" + j.split("|")[1]
    response = request.urlopen(url)
    soup = BeautifulSoup(response)
    response.close()
    
    katamari = soup.find_all('td', class_='featdescription')
    
    l_d = []
    
    for i in katamari:
        domain = i.find('span', property='text')
        if domain == None:
            pass
        elif len(domain.text) > 40:
            pass
        else:
            l_d.append(domain.text)
    return l_d

In [31]:
tqdm_notebook.pandas(desc="Progress: ")
### エラー５０３に気をつける ###
df1["Domain"] = df1["OS"].progress_apply(DomainName)

UniprotのTaxonomyを検索してTaxonomyを取得する

In [33]:
df1.to_csv('argonaute_final_domain.tsv', sep='\t', index=True)

In [213]:
def url_check(url):
    try:
        res = request.urlopen(url)
    except error.HTTPError as e:
        return e.code
    except RemoteDisconnected:
        return 1000

def Taxonomy(k):
    url = "https://www.uniprot.org/taxonomy/?query=" + k.replace(" ","+") + "&sort=score"
    if url_check(url)==400 or url_check(url)==404:
        k.split(" ")
        k = '+'.join(k.split(" ")[:-2])
        url = "https://www.uniprot.org/taxonomy/?query=" + k + "&sort=score"
        if url_check(url) == 1000 or url_check(url) == 502:
            print(k," Error")
            return "Error"
    elif url_check(url) == 1000 or url_check(url) == 502:
        print(k," Error")
        return "Error"
    response = request.urlopen(url)
    soup = BeautifulSoup(response)
    response.close()
    if "Sorry" in soup.text:
        k.split(" ")
        k = '+'.join(k.split(" ")[:-2])
        url = "https://www.uniprot.org/taxonomy/?query=" + k + "&sort=score"
        if url_check(url) == 1000 or url_check(url) == 502:
            print(k," Error")
            return "Error"
        response = request.urlopen(url)
        soup = BeautifulSoup(response)
        response.close()
    l_tax = soup.find('p', class_='summary').text
    return l_tax

エラー５０３に気をつける。
Remote end closed connection without responseはやり直す

In [220]:
df1["Taxonomy"] = df1["OX"]

In [240]:
for i in range(136,150):
    tqdm_notebook.pandas(desc="Progress "+str(i)+" : ")
    df1["Taxonomy"][i*100:(i+1)*100] = df1["OX"][i*100:(i+1)*100].progress_apply(Taxonomy)

/home/pacbio/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [242]:
df1["Taxonomy"][15000:] = df1["OX"][15000:].progress_apply(Taxonomy)

/home/pacbio/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Errorのやり直し

In [247]:
df1[df1["Taxonomy"]=="Error"]["OX"].progress_apply(Taxonomy)

41       Eukaryota › Metazoa › Chordata › Craniata › Ve...
5412     Eukaryota › Viridiplantae › Streptophyta › Emb...
5414     Eukaryota › Viridiplantae › Streptophyta › Emb...
12819    Eukaryota › Metazoa › Chordata › Craniata › Ve...
13239    Eukaryota › Metazoa › Chordata › Craniata › Ve...
Name: OX, dtype: object

In [256]:
df1["COREaa"] = df["COREaa"]

In [258]:
df1.to_csv('argonaute_final_domain_taxonomy.tsv', sep='\t', index=True)